# Voronoi Image
Demo the Voronoi image code

In [ ]:
%load_ext snakeviz
%load_ext autoreload
%autoreload 2
import numpy as np
import matplotlib.pyplot as plt
import IPython.display as ipd
import skimage.io
import time

import sys
sys.path.append("../src")
from imtools import get_voronoi_image

In [ ]:
## Parameters
device = 'cuda'
n_points = 5000
n_iters = 100
I = skimage.io.imread("../data/images/layla.png")
print("Compression ratio", I.shape[0]*I.shape[1]/n_points)

In [ ]:
tic = time.time()
J, xy, rgb, final_cost = get_voronoi_image(I, device, n_points, n_neighbs=2, n_iters=n_iters, 
                                           verbose=True, plot_iter_interval=0, use_lsqr=False)
print("Elapsed Time: {:.3f}".format(time.time()-tic))
plt.imshow(J)
plt.title("Final cost {:.3f}".format(final_cost))
plt.savefig("Final.png", bbox_inches='tight')